In [1]:
#!/usr/bin/env python
#coding=utf-8


In [2]:
import config

accesskey_id = config.alicloud_accesskey_id
accesskey_secret = config.alicloud_accesskey_secret
region_id = config.alicloud_region_id

projectname = config.alicloud_dataworks_projectname
flowname = config.alicloud_dataworks_flowname
bizdate = config.alicloud_dataworks_bizdate
bizdate = config.alicloud_dataworks_bizdate

oss_bucket = config.alicloud_oss_bucket
oss_endpoint = config.alicloud_oss_endpoint

http_root = 'http://%s.%s/'%(oss_bucket, oss_endpoint)
print(http_root)

http://demopageranker.oss-eu-central-1.aliyuncs.com/


# Preparation 1: function for oss

In [3]:
import oss2 # pip install oss2
import os
import datetime

def oss_check_fn(prefix):
    result = bucket.list_objects(prefix=prefix, delimiter='', marker='', max_keys=100)
    fn = ''
    ts = ''
    for r in result.object_list:
        #print(r.key)
        if ts == '':
            ts = r.last_modified
        if ts >= r.last_modified:
            ts = r.last_modified
            fn = r.key
            bucket.put_object_acl(fn, oss2.BUCKET_ACL_PUBLIC_READ)
            
    return fn

def oss_validate_fn(key):
    result = bucket.get_object(key)
    
    with open(key, "rb") as binary_file:
        # Read the whole file at once
        content = binary_file.read()

    content_got = b''
    for chunk in result:
        content_got += chunk
    assert content_got == content
    
    filename = 'download.txt'
    result = bucket.get_object_to_file(key, filename)
    
    with open(filename, 'rb') as fileobj:
        assert fileobj.read() == content
        

def oss_upload_fn(key):
    with open(key, "rb") as binary_file:
        # Read the whole file at once
        content = binary_file.read()
    bucket.put_object(key, content, headers={})
    bucket.put_object_acl(key, oss2.BUCKET_ACL_PUBLIC_READ)
    

def oss_check_out_fn(prefix):
    fn = '%s%s'%(http_root, oss_check_fn(prefix))
    return fn

In [4]:
# Preparation 2: function for dataworks

In [5]:
import json
from pprint import pprint
import datetime
import time

from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.request import CommonRequest

def dataworks_api_create_adhoc_dag(accesskey_id, accesskey_secret, region_id, projectname, flowname, bizdate='2018-07-25 00:00:00'):
    client = AcsClient(accesskey_id, accesskey_secret, region_id)

    request = CommonRequest()
    request.set_accept_format('json')
    request.set_domain('dataworks.aliyuncs.com')
    request.set_method('POST')
    request.set_protocol_type('https') # https | http
    request.set_version('2018-06-01')
    request.set_action_name('CreateManualDag')
    
    request.add_query_param('ProjectName', projectname)
    request.add_query_param('FlowName', flowname)
    request.add_query_param('Bizdate', bizdate)

    response = client.do_action(request)
    #pprint(response)
    js = json.loads(response)
    status = 'Failed and unknown'
    dag_id = '0'
    if ('ReturnCode' in js):
        if (js['ReturnCode'] == '600011'):
            status = 'Failed and Bizdate should be in [2019-03-20 00:00:00 format]'
        if (js['ReturnCode'] == '0') and ('ReturnValue' in js): 
            status = 'Succeed'
            dag_id = js['ReturnValue']
    return js, status, dag_id
    
    
def dataworks_api_check_dag(accesskey_id, accesskey_secret, region_id, projectname, flowname, dag_id):
    client = AcsClient(accesskey_id, accesskey_secret, region_id)

    request = CommonRequest()
    request.set_accept_format('json')
    request.set_domain('dataworks.aliyuncs.com')
    request.set_method('POST')
    request.set_protocol_type('https') # https | http
    request.set_version('2018-06-01')
    request.set_action_name('SearchManualDagNodeInstance')

    request.add_query_param('ProjectName', projectname)
    request.add_query_param('DagId', dag_id)

    response = client.do_action(request)
    js = json.loads(response)
    status = 'Failed and unknown'
    if 'ErrCode' in js:
        if js['ErrCode'] == '11020293069':
            status = 'dag_id is not validated'
        if js['ErrCode'] == '0':
            status = 'Succeed'
    return js, status

def check_dag_status(status_code):
    dag_status = {
        1: {'en': 'NOT_RUN', 'cn': "未运行"},
        2: {'en': 'WAIT_TIME', 'cn': "等待时间"},
        3: {'en': 'WAIT_RESOURCE', 'cn': "等待资源"},
        4: {'en': 'RUNNING', 'cn': "运行中"},
        5: {'en': 'FAILURE', 'cn': "运行失败"},
        6: {'en': 'SUCCESS', 'cn': "运行成功"},
        7: {'en': 'CHECKING', 'cn': "校验中"}
    }
    if status_code in dag_status:
        return dag_status[status_code]
    else:
        return 'invalidate status_code, please check'
    
def dt_to_ds(dt):
    dt = int(dt)
    ds = datetime.datetime.fromtimestamp(dt / 1e3)
    return ds

def wait_for_dag_to_complete(accesskey_id, accesskey_secret, region_id, projectname, flowname, dag_id):
    while True:
        js, status = dataworks_api_check_dag(accesskey_id, accesskey_secret, region_id, projectname, flowname, dag_id)
        dag_finish = 1
        if ('Data' in js) and ('NodeInsInfo' in js['Data']):
            for node in js['Data']['NodeInsInfo']:
                dag_id = node['DagId']
                biz_date = node['Bizdate']
                instance_id = node['InstanceId']
                create_time = node['CreateTime']
                modify_time = node['ModifyTime']
                status = check_dag_status(node['Status'])
                print('%s | %s | %s \n | %s | %s | %s'%(dag_id, instance_id, status, dt_to_ds(biz_date), dt_to_ds(create_time), dt_to_ds(modify_time)))
                if node['Status'] != 6:
                    dag_finish = 0
        if dag_finish == 1:
            print("** whole dag is completed, please go to oss to fetch latest file **")
            break
        else:
            print("**** please wait for the dag to be complete.... ****")
            time.sleep(5)

            #js, status = dataworks_api_check_dag(accesskey_id, accesskey_secret, region_id, projectname, flowname, instance_id)
            #pprint(js)
    return 


# Step 1: upload a source CSV from local to oss

In [6]:
bucket = oss2.Bucket(oss2.Auth(accesskey_id, accesskey_secret), oss_endpoint, oss_bucket)
print(bucket)

key = 'in/slack_example_with_head.csv'
oss_upload_fn(key)

fn = '%s%s'%(http_root, oss_check_fn(key))
print(fn)

oss_validate_fn(key)

import requests
r = requests.get(fn)
print(r.text)

http://demopageranker.oss-eu-central-1.aliyuncs.com/in/slack_example_with_head.csv
"Adam Cook","Deangelo Giraldo",1
"Adam Cook","Rashford Kane",2
"Adele Robert","Adele Robert",2
"Adele Robert","Chuck Alex",4
"Adele Robert","Jamie Rooney",1
"Adele Robert","Kyle Stones",2
"Andy McClaren","Bradley Bam",17
"Andy McClaren","David Hinks",3
"Andy McClaren","Joe Trippier",10
"Andy McClaren","Kyle Stones",20
"Andy McClaren","Rueben Heidel",30
"Andy McClaren","Wanda Walker",2
"Bradley Bam","Andy McClaren",30
"Bradley Bam","Bradley Bam",1
"Bradley Bam","David Hinks",2
"Bradley Bam","Joe Trippier",96
"Bradley Bam","John Brandon",35
"Bradley Bam","Jordan Butland",2
"Bradley Bam","Kyle Stones",267
"Bradley Bam","Marcel Upamecano",2
"Bradley Bam","Mosley Chris",1
"Bradley Bam","Rueben Heidel",207
"Bradley Bam","Wanda Walker",54
"Chuck Alex","Adele Robert",43
"Chuck Alex","Chuck Alex",75
"Chuck Alex","Jamie Rooney",40
"Chuck Alex","Jordan Butland",2
"Chuck Alex","Kyle Stones",73
"Chuck Alex","Ruben Li

# Step 2: create DAG and wait it for finish in dataworks

In [7]:
js, status, dag_id = dataworks_api_create_adhoc_dag(accesskey_id, accesskey_secret, region_id, projectname, flowname, bizdate)

print('api_return: ')
pprint(js)
print('status: ',  status)
print('dag_id: ', dag_id)


api_return: 
{'RequestId': '476D6A2B-D74C-4DF7-A3FC-A66B46AD5C18',
 'ReturnCode': '0',
 'ReturnErrorSolution': '',
 'ReturnMessage': '',
 'ReturnValue': 700021546810}
status:  Succeed
dag_id:  700021546810


In [8]:
js, status = dataworks_api_check_dag(accesskey_id, accesskey_secret, region_id, projectname, flowname, dag_id)

print('api_return: ')
pprint(js)



api_return: 
{'Data': {'NodeInsInfo': [{'BeginWaitTimeTime': 1554480830049,
                           'Bizdate': 1553702400000,
                           'CreateTime': 1554480829986,
                           'DagId': 700021546810,
                           'DagType': 5,
                           'InstanceId': 705785096913,
                           'ModifyTime': 1554480830049,
                           'NodeName': 'vn_start',
                           'Status': 2},
                          {'Bizdate': 1553702400000,
                           'CreateTime': 1554480829943,
                           'DagId': 700021546810,
                           'DagType': 5,
                           'InstanceId': 705785096909,
                           'ModifyTime': 1554480829943,
                           'NodeName': 'ddl_ods_pagerank',
                           'Status': 1},
                          {'Bizdate': 1553702400000,
                           'CreateTime': 1554480829952,
 

In [9]:
wait_for_dag_to_complete(accesskey_id, accesskey_secret, region_id, projectname, flowname, dag_id)

700021546810 | 705785096913 | {'en': 'WAIT_TIME', 'cn': '等待时间'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.049000
700021546810 | 705785096909 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.943000 | 2019-04-05 17:13:49.943000
700021546810 | 705785096910 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.952000 | 2019-04-05 17:13:49.952000
700021546810 | 705785096911 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.962000 | 2019-04-05 17:13:49.962000
700021546810 | 705785096912 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.977000 | 2019-04-05 17:13:49.977000
**** please wait for the dag to be complete.... ****
700021546810 | 705785096913 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.485000
700021546810 | 705785096909 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 

700021546810 | 705785096913 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.485000
700021546810 | 705785096909 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.943000 | 2019-04-05 17:13:49.943000
700021546810 | 705785096910 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.952000 | 2019-04-05 17:13:49.952000
700021546810 | 705785096911 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.962000 | 2019-04-05 17:13:49.962000
700021546810 | 705785096912 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.977000 | 2019-04-05 17:13:49.977000
**** please wait for the dag to be complete.... ****
700021546810 | 705785096913 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.485000
700021546810 | 705785096909 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 

700021546810 | 705785096913 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.485000
700021546810 | 705785096909 | {'en': 'RUNNING', 'cn': '运行中'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.943000 | 2019-04-05 17:14:58.417000
700021546810 | 705785096910 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.952000 | 2019-04-05 17:13:49.952000
700021546810 | 705785096911 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.962000 | 2019-04-05 17:13:49.962000
700021546810 | 705785096912 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.977000 | 2019-04-05 17:13:49.977000
**** please wait for the dag to be complete.... ****
700021546810 | 705785096913 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.485000
700021546810 | 705785096909 | {'en': 'RUNNING', 'cn': '运行中'} 
 | 2019-03-27 16:00:00 | 

700021546810 | 705785096913 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.485000
700021546810 | 705785096909 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.943000 | 2019-04-05 17:16:01.080000
700021546810 | 705785096910 | {'en': 'RUNNING', 'cn': '运行中'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.952000 | 2019-04-05 17:16:09.221000
700021546810 | 705785096911 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.962000 | 2019-04-05 17:13:49.962000
700021546810 | 705785096912 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.977000 | 2019-04-05 17:13:49.977000
**** please wait for the dag to be complete.... ****
700021546810 | 705785096913 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.485000
700021546810 | 705785096909 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 

700021546810 | 705785096913 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.485000
700021546810 | 705785096909 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.943000 | 2019-04-05 17:16:01.080000
700021546810 | 705785096910 | {'en': 'RUNNING', 'cn': '运行中'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.952000 | 2019-04-05 17:16:09.221000
700021546810 | 705785096911 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.962000 | 2019-04-05 17:13:49.962000
700021546810 | 705785096912 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.977000 | 2019-04-05 17:13:49.977000
**** please wait for the dag to be complete.... ****
700021546810 | 705785096913 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.485000
700021546810 | 705785096909 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 

700021546810 | 705785096913 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.485000
700021546810 | 705785096909 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.943000 | 2019-04-05 17:16:01.080000
700021546810 | 705785096910 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.952000 | 2019-04-05 17:17:38.479000
700021546810 | 705785096911 | {'en': 'RUNNING', 'cn': '运行中'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.962000 | 2019-04-05 17:17:49.456000
700021546810 | 705785096912 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.977000 | 2019-04-05 17:13:49.977000
**** please wait for the dag to be complete.... ****
700021546810 | 705785096913 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.485000
700021546810 | 705785096909 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00

700021546810 | 705785096913 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.485000
700021546810 | 705785096909 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.943000 | 2019-04-05 17:16:01.080000
700021546810 | 705785096910 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.952000 | 2019-04-05 17:17:38.479000
700021546810 | 705785096911 | {'en': 'RUNNING', 'cn': '运行中'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.962000 | 2019-04-05 17:17:49.456000
700021546810 | 705785096912 | {'en': 'NOT_RUN', 'cn': '未运行'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.977000 | 2019-04-05 17:13:49.977000
**** please wait for the dag to be complete.... ****
700021546810 | 705785096913 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.485000
700021546810 | 705785096909 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00

700021546810 | 705785096913 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.485000
700021546810 | 705785096909 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.943000 | 2019-04-05 17:16:01.080000
700021546810 | 705785096910 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.952000 | 2019-04-05 17:17:38.479000
700021546810 | 705785096911 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.962000 | 2019-04-05 17:20:00.435000
700021546810 | 705785096912 | {'en': 'RUNNING', 'cn': '运行中'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.977000 | 2019-04-05 17:20:18.885000
**** please wait for the dag to be complete.... ****
700021546810 | 705785096913 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:00 | 2019-04-05 17:13:49.986000 | 2019-04-05 17:13:50.485000
700021546810 | 705785096909 | {'en': 'SUCCESS', 'cn': '运行成功'} 
 | 2019-03-27 16:00:0

# Step 3: get output file address to curl

In [10]:
bucket = oss2.Bucket(oss2.Auth(accesskey_id, accesskey_secret), oss_endpoint, oss_bucket)
print(bucket)

prefix='out/dwd_'
fn = oss_check_out_fn(prefix)
print(fn)

import requests
r = requests.get(fn)
print(r.text)

http://demopageranker.oss-eu-central-1.aliyuncs.com/out/dwd_pagerank__983b6803bc804c1c91a5a0d662e83913
Jordan Butland,0.01313744,2019-04-06 00:19:50
Rashford Kane,0.01833751,2019-04-06 00:19:50
Young Marshal,0.01555849,2019-04-06 00:19:50
John Brandon,0.02008903,2019-04-06 00:19:50
Jamie Rooney,0.02290454,2019-04-06 00:19:50
Gary Carew,0.0219757,2019-04-06 00:19:50
Adele Robert,0.02397198,2019-04-06 00:19:50
Lewis Jordan,0.00839385,2019-04-06 00:19:50
Rueben Heidel,0.18135253,2019-04-06 00:19:50
Reid Keith,0.01430576,2019-04-06 00:19:50
Raheem Eric,0.00941539,2019-04-06 00:19:50
Callum Sancho,0.00839385,2019-04-06 00:19:50
Nathaniel Mount,0.00870665,2019-04-06 00:19:50
Ryan Stephen,0.00828689,2019-04-06 00:19:50
Adam Cook,0.00962527,2019-04-06 00:19:50
Lewis Dier,0.00849879,2019-04-06 00:19:50
Bradley Bam,0.09798232,2019-04-06 00:19:50
Jerry Micah,0.00828689,2019-04-06 00:19:50
Joe Trippier,0.04984763,2019-04-06 00:19:50
Maria Tadic,0.01070607,2019-04-06 00:19:50
Tavon Kenny,0.00870665